In [1]:
import os, sys
sys.path = list(set([
    "../../lib/",
] + sys.path))
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Iterable, Any
from plotly import graph_objects as go, subplots as sp
from concurrent.futures import ProcessPoolExecutor as Exe
import networkx as nx
# import numba

from local.caching import load, save, save_exists
from local.figures import layout, xaxis_desc, yaxis_desc

In [2]:
# change in Q for each v in cluster if moved to other
def DeltaQ(G: nx.Graph, cluster, other):
    degrees_within = [G.degree[u] for u in cluster]
    degrees_out = [G.degree[u] for u in other]
    sum_in = np.sum(degrees_within)
    sum_out = np.sum(degrees_out)

    for v in cluster:
        neighbours = [u for u in G.neighbors(v)]
        if len(other) > 0 and all(u in cluster for u in neighbours): continue
        edges_within = [u for u in neighbours if u in cluster]
        edges_out = [u for u in neighbours if u not in cluster]

        vsum_in = (sum_in - G.degree[v])
        vsum_out = sum_out
        Q = (len(edges_out) - len(edges_within)) / len(G.edges)
        Q -= G.degree[v] * (vsum_out - vsum_in) / (2 * len(G.edges)**2)

        yield Q, v

def ToGive(G: nx.Graph, cluster: set[int], other: set[int]):
    if len(cluster)==0: return [], []
    to_give = sorted(DeltaQ(G, cluster, other), key=lambda t: t[0], reverse=True)
    if len(cluster) == len(G):
        cut = 1
        force = True
    else:
        cut = max(1, len(G)//100)
        # cut = 1
        force = False 

    qs, vs = [], []
    for q, v in to_give:
        if len(qs) >= cut: break
        if not force and q <= 0: continue
        qs.append(q)
        vs.append(v)
    return qs, vs

def Partition(G: nx.Graph):
    if len(G.edges) == 0: return None

    cluster = {v for v in G.nodes}
    other = set()

    i = 0
    delta_q = 0
    _break = False
    while not _break:
        qs, to_give = ToGive(G, cluster, other)
        if len(to_give) == 0: break
        if len(to_give) >= len(cluster):
            cut = len(cluster)-1
            qs = qs[:cut]
            to_give = to_give[:cut]
            _break = True
        delta_q += np.sum(qs)
        cluster = cluster.difference(to_give)
        other = other.union(to_give)
        i += 1
    # print(f"iterations: {i}, deltaQ: {delta_q}, {len(cluster)}, {len(other)}")
    if len(other) == 0: return None

    def _partition(cluster: set[int]):
        g = nx.Graph()
        g.add_nodes_from(cluster)
        for v, u in G.edges:
            if v in cluster and u in cluster:
                g.add_edge(v, u)
        return g

    return _partition(cluster), _partition(other), delta_q

def Cluster(G: nx.Graph):
    _original = G
    clusters = [{v for v in G.nodes}]
    current_modularity = nx.algorithms.community.modularity(G, clusters)
    def _is_same(a: set, b: set):
        return next(iter(a)) in b
    
    def _cluster(G: nx.Graph, optimal=True):
        leaf = [v for v in G.nodes]
        if len(G) <= 1: return leaf
        part = Partition(G)
        if part is None: return leaf

        a, b, dq = part

        if optimal:
            ab = {v for v in G.nodes}
            cluster_a = {v for v in a}
            cluster_b = {v for v in b}
            if len(cluster_a) > len(cluster_b):
                inplace, new = cluster_a, cluster_b
            else:
                inplace, new = cluster_b, cluster_a

            nonlocal clusters, current_modularity
            snapshot = []
            for c in clusters:
                if  _is_same(c, ab): 
                    snapshot.append(inplace)
                else:
                    snapshot.append(c)
            snapshot.append(new)
            new_mod = nx.algorithms.community.modularity(_original, snapshot)
            if new_mod < current_modularity:
                optimal = False
            else:
                current_modularity = new_mod
                clusters = snapshot

        return _cluster(a, optimal), _cluster(b, optimal), dq
    
    return _cluster(G), clusters, current_modularity

In [3]:
from txyl_common.biocyc_facade.pgdb import Pgdb, Dat, Traceable

np.random.seed(123)
pgdb_dir = Path("../../data/txyl_local/biocyc/pgdbs")
pgdb_files = list(os.listdir(pgdb_dir))
sample_file = pgdb_files[np.random.randint(0, len(pgdb_files))]
pgdb = Pgdb(pgdb_dir.joinpath(sample_file))
_info = pgdb.GetInfo()
for k in "PGDB-NAME, ORGANISM, NCBI-TAXONOMY-ID, PGDB-TIER, biocyc_facade_ver, Total_entries".split(", "):
    v = _info[k].replace('"', "")
    print(f"{k}:{' '*(25-len(k))}{v}")

PGDB-NAME:                gcf_001563605cyc
ORGANISM:                 Corynebacterium sp. CMW7794
NCBI-TAXONOMY-ID:         SAMN03850947
PGDB-TIER:                3
biocyc_facade_ver:        1.3
Total_entries:            12917


In [4]:
class MNetwork:
    def __init__(self, reactions: dict) -> None:
        ccode = {}
        def encode(c):
            if c not in ccode: 
                ccode[c] = len(ccode)
            return ccode[c]

        clookup = {}
        rlookup = {}
        for k, v in reactions.items():
            lefts = v.get("LEFT", [])
            rights = v.get("RIGHT", [])
            k = encode(k)

            cpds = set(lefts).union(rights)
            rlookup[k] = {encode(c) for c in cpds}
            for c in cpds:
                ci = encode(c)
                ref = clookup.get(ci, set())
                ref.add(k)
                clookup[ci] = ref

        G = nx.Graph()
        for v, rxns in clookup.items():
            for rxn in rxns:
                G.add_edge(v, rxn)

        for rxn, cpds in rlookup.items():
            for u in cpds:
                G.add_edge(rxn, u)

        self.G = G
        self._ccode = ccode
        revcode = ['']*len(ccode)
        for k, i in ccode.items():
            revcode[i] = k
        self._revcode = revcode

    def Decode(self, index: int):
        return self._revcode[index] if index < len(self._revcode) else None
    
    def Encode(self, cpd: str):
        return self._ccode.get(cpd)
    
net = MNetwork(pgdb.GetDataTable(Dat.REACTIONS))

In [5]:
tree, clusters, mod = Cluster(net.G)

In [6]:
[f"{i}:{len(c)}" for i, c in enumerate(clusters)]

['0:504',
 '1:536',
 '2:62',
 '3:392',
 '4:186',
 '5:6',
 '6:58',
 '7:32',
 '8:30',
 '9:5',
 '10:158',
 '11:43',
 '12:100',
 '13:61',
 '14:50',
 '15:31',
 '16:18',
 '17:208',
 '18:72',
 '19:38']

In [8]:
cpds = pgdb.GetDataTable(Dat.COMPOUNDS)
cnames = dict(pgdb.Trace(Traceable.COMPOUNDS, Traceable.CMPD_COMMON_NAME))
for i, c in enumerate(clusters):
    if i != 14: continue
    print(len(c))

    cids = [n for n in [net.Decode(x) for x in c] if n in cpds]
    for id in cids:
        # print(f"{id} -- {cnames.get(id, id)}")
        print(id if not id.startswith("CPD") else cnames.get(id, id))

    break

50
Octanoylated-domains
methanethiol
HS
ACETYLSERINE
hydrogen selenide
ACET
L-SELENOCYSTEINE
O-SUCCINYL-L-HOMOSERINE
Mercapturates
a mycothiol <i>S</i>-conjugate
hydrosulfide
acetic acid
HOMO-CYS
5-METHYL-THF-GLU-N
METHIONINE-SYNTHASE-METHYLCOBALAMIN
Methionine-synthase-cob-I-alamins
S<sup>2-</sup>
Dihydro-Lipoyl-Proteins
tetrahydropteroyl tri-L-glutamate
5-methyltetrahydropteroyl tri-L-glutamate
SELENOMETHIONINE
SELENOHOMOCYSTEINE
<i>O</i>-acetyl-L-homoserine
HOMO-SER
1-(2-amino-2-deoxy-&alpha;-D-glucopyranosyl)-1D-myo-inositol
ACETAMIDE
